# Lab 5: Dry Beans Classification

Jacob Ballard  
Jack Bradley  
DSC 305a S22  
Lab 5: Dry Beans Classification  

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_excel("Dry_Bean_Dataset.xlsx")
df


,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653248,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [3]:
df.Class = pd.Categorical(df.Class)
df.Class = df.Class.cat.codes
df.Class

0        5
1        5
2        5
3        5
4        5
        ..
13606    3
13607    3
13608    3
13609    3
13610    3
Name: Class, Length: 13611, dtype: int8

In [4]:
df.describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
count,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000,13611.000000
mean,53048.284549,855.283459,320.141867,202.270714,1.583242,0.750895,53768.200206,253.064220,0.749733,0.987143,0.873282,0.799864,0.006564,0.001716,0.643590,0.995063,3.532657
std,29324.095717,214.289696,85.694186,44.970091,0.246678,0.092002,29774.915817,59.177120,0.049086,0.004660,0.059520,0.061713,0.001128,0.000596,0.098996,0.004366,1.828709
min,20420.000000,524.736000,183.601165,122.512653,1.024868,0.218951,20684.000000,161.243764,0.555315,0.919246,0.489618,0.640577,0.002778,0.000564,0.410339,0.947687,0.000000
25%,36328.000000,703.523500,253.303633,175.848170,1.432307,0.715928,36714.500000,215.068003,0.718634,0.985670,0.832096,0.762469,0.005900,0.001154,0.581359,0.993703,2.000000
50%,44652.000000,794.941000,296.883367,192.431733,1.551124,0.764441,45178.000000,238.438026,0.759859,0.988283,0.883157,0.801277,0.006645,0.001694,0.642044,0.996386,3.000000
75%,61332.000000,977.213000,376.495012,217.031741,1.707109,0.810466,62294.000000,279.446467,0.786851,0.990013,0.916869,0.834270,0.007271,0.002170,0.696006,0.997883,5.000000
max,254616.000000,1985.370000,738.860153,460.198497,2.430306,0.911423,263261.000000,569.374358,0.866195,0.994677,0.990685,0.987303,0.010451,0.003665,0.974767,0.999733,6.000000


In [5]:
for f in df.columns:
    print(str(df[f].describe()))

count     13611.000000
mean      53048.284549
std       29324.095717
min       20420.000000
25%       36328.000000
50%       44652.000000
75%       61332.000000
max      254616.000000
Name: Area, dtype: float64
count    13611.000000
mean       855.283459
std        214.289696
min        524.736000
25%        703.523500
50%        794.941000
75%        977.213000
max       1985.370000
Name: Perimeter, dtype: float64
count    13611.000000
mean       320.141867
std         85.694186
min        183.601165
25%        253.303633
50%        296.883367
75%        376.495012
max        738.860153
Name: MajorAxisLength, dtype: float64
count    13611.000000
mean       202.270714
std         44.970091
min        122.512653
25%        175.848170
50%        192.431733
75%        217.031741
max        460.198497
Name: MinorAxisLength, dtype: float64
count    13611.000000
mean         1.583242
std          0.246678
min          1.024868
25%          1.432307
50%          1.551124
75%          1.707109

In [6]:
df.dtypes

Area                 int64
Perimeter          float64
MajorAxisLength    float64
MinorAxisLength    float64
AspectRation       float64
Eccentricity       float64
ConvexArea           int64
EquivDiameter      float64
Extent             float64
Solidity           float64
roundness          float64
Compactness        float64
ShapeFactor1       float64
ShapeFactor2       float64
ShapeFactor3       float64
ShapeFactor4       float64
Class                 int8
dtype: object

In [7]:
df[df.duplicated()]

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
5505,33518,702.956,277.571399,154.305581,1.798842,0.831240,34023,206.582775,0.808383,0.985157,0.852377,0.744251,0.008281,0.001567,0.553909,0.996396,4
5509,33954,716.750,277.368480,156.356326,1.773951,0.825970,34420,207.922042,0.799482,0.986461,0.830549,0.749624,0.008169,0.001591,0.561936,0.996847,4
5548,38427,756.323,306.533886,160.591784,1.908777,0.851782,38773,221.193978,0.796976,0.991076,0.844174,0.721597,0.007977,0.001334,0.520702,0.993905,4
5554,38891,791.343,319.499996,156.869619,2.036723,0.871168,39651,222.525412,0.650025,0.980833,0.780422,0.696480,0.008215,0.001192,0.485085,0.987983,4
5599,40804,790.802,323.475648,163.287717,1.981016,0.863241,41636,227.932592,0.787570,0.980017,0.819931,0.704636,0.007928,0.001206,0.496512,0.983598,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7263,63408,1005.966,412.551649,196.337705,2.101235,0.879494,64200,284.136539,0.798791,0.987664,0.787385,0.688730,0.006506,0.000903,0.474348,0.996718,4
7278,63882,1004.206,411.263403,198.765453,2.069089,0.875452,64663,285.196579,0.754705,0.987922,0.796054,0.693465,0.006438,0.000918,0.480893,0.995010,4
7285,63948,996.497,412.297178,198.877557,2.073121,0.875971,64641,285.343867,0.777909,0.989279,0.809254,0.692083,0.006447,0.000912,0.478979,0.992981,4
7340,65766,1035.842,406.416622,207.242369,1.961069,0.860218,66698,289.371512,0.792295,0.986027,0.770237,0.712007,0.006180,0.000980,0.506954,0.994172,4


In [8]:
df = df.drop_duplicates()
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,5
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,5
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,5
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,5
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,3
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,3
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,3
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653248,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,3


In [9]:
y = df.Class
X = df.iloc[:, :-1]
y

0        5
1        5
2        5
3        5
4        5
        ..
13606    3
13607    3
13608    3
13609    3
13610    3
Name: Class, Length: 13543, dtype: int8

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y,
                                                    test_size=0.25, 
                                                    random_state=2022)

In [11]:
X_train.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
679,37438,698.989,238.793873,199.754991,1.195434,0.547942,37738,218.328977,0.776223,0.992050,0.962901,0.914299,0.006378,0.002749,0.835942,0.999314
7770,39595,736.146,273.683299,184.932773,1.479907,0.737160,39975,224.530443,0.779767,0.990494,0.918168,0.820402,0.006912,0.001932,0.673060,0.996066
8803,44838,807.043,300.736509,190.773486,1.576406,0.773042,45454,238.934122,0.824379,0.986448,0.865093,0.794497,0.006707,0.001648,0.631225,0.995067
142,33731,671.696,231.172013,186.126456,1.242016,0.593081,34073,207.238131,0.803215,0.989963,0.939493,0.896467,0.006853,0.002730,0.803653,0.998150
3001,75871,1061.687,361.055665,268.946659,1.342481,0.667188,76977,310.808554,0.808514,0.985632,0.845849,0.860833,0.004759,0.001612,0.741033,0.994823


In [12]:
y_train.head()

679     5
7770    6
8803    6
142     5
3001    0
Name: Class, dtype: int8

In [13]:
pipe = Pipeline(steps=[('scaler', StandardScaler()),
                       ('classifier', LogisticRegression(solver='lbfgs', max_iter=1000))])

In [14]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression(max_iter=1000))])

In [15]:
grid_param = [
#     {"classifier": [LogisticRegression(max_iter=1000)],
#      "classifier__penalty": ['l2','l1'],
#      "classifier__C": np.logspace(-4, 4, 9),
#      "classifier__solver": ['saga'],
#      "classifier__random_state": [2022]},
    
    {"classifier": [RandomForestClassifier()],
     "classifier__n_estimators": [10, 100, 1000],
     "classifier__max_depth":[5, 8, 15, 25, None],
     "classifier__min_samples_leaf":[2, 5, 10, 15, 100],
     "classifier__max_leaf_nodes": [2, 5, 10, 15],
     "classifier__random_state": [2022]}, 
    
    {"classifier": [KNeighborsClassifier()],
     "classifier__n_neighbors": [3, 5],
     "classifier__metric": ['manhattan']},
    
    {"classifier": [DecisionTreeClassifier()],
     "classifier__random_state" : [2022]},
    
    {"classifier": [RandomForestClassifier()],
     "classifier__n_estimators": [5, 10, 15],
     "classifier__random_state": [2022]},
    
    {"classifier": [SVC()],
     "classifier__probability": [True],
     "classifier__random_state": [2022]}
    
]

In [20]:
gridsearch = GridSearchCV(pipe, grid_param, cv=5, scoring='f1_macro', 
                          verbose=1, n_jobs=-1) # Fit grid search
best_model = gridsearch.fit(X_train,y_train)

Fitting 5 folds for each of 307 candidates, totalling 1535 fits


In [21]:
best_model.best_params_

{'classifier': SVC(probability=True, random_state=2022),
 'classifier__probability': True,
 'classifier__random_state': 2022}

In [22]:
best_model.score(X_test, y_test)

0.9396046538522886

In [23]:
y_predict_proba = best_model.predict_proba(X_test)

In [24]:
y_predict_proba[:5, :]

array([[3.06818119e-07, 2.10850474e-07, 2.19041949e-07, 9.99094526e-01,
        3.40547228e-04, 6.93515236e-06, 5.57254575e-04],
       [6.29717069e-04, 5.86471153e-05, 3.91834194e-04, 2.42169486e-02,
        1.67384968e-03, 2.80099960e-03, 9.70228004e-01],
       [5.74811222e-01, 6.41824005e-05, 4.24224151e-01, 8.56461458e-06,
        2.77384381e-04, 2.14070900e-04, 4.00424550e-04],
       [9.35403456e-04, 3.84682441e-05, 1.39024553e-04, 4.56555656e-02,
        1.97276664e-03, 1.05316848e-03, 9.50205603e-01],
       [1.24907600e-04, 9.42296107e-05, 1.27507728e-04, 6.23441969e-03,
        3.95583378e-04, 9.92566458e-01, 4.56894270e-04]])

In [25]:
y_predict = (y_predict_proba[:,1] > 0.25).astype(int)

In [26]:
y_predict[:5]

array([0, 0, 0, 0, 0])

In [28]:
results = pd.DataFrame(best_model.cv_results_)
results.sort_values(by='rank_test_score', inplace= True)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__max_depth,param_classifier__max_leaf_nodes,param_classifier__min_samples_leaf,param_classifier__n_estimators,param_classifier__random_state,...,param_classifier__probability,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
306,6.149136,0.875740,0.664061,0.110663,"SVC(probability=True, random_state=2022)",NaN,NaN,NaN,NaN,2022,...,True,"{'classifier': SVC(probability=True, random_st...",0.938160,0.945715,0.938014,0.943129,0.936509,0.940305,0.003507,1
301,0.015886,0.003403,1.408468,0.188250,KNeighborsClassifier(),NaN,NaN,NaN,NaN,NaN,...,NaN,"{'classifier': KNeighborsClassifier(), 'classi...",0.931173,0.939891,0.931457,0.931630,0.924525,0.931735,0.004877,2
305,0.968713,0.391366,0.019567,0.003549,RandomForestClassifier(),NaN,NaN,NaN,15,2022,...,NaN,"{'classifier': RandomForestClassifier(), 'clas...",0.926365,0.934216,0.938886,0.928343,0.925747,0.930711,0.005065,3
304,0.442521,0.012808,0.013396,0.001060,RandomForestClassifier(),NaN,NaN,NaN,10,2022,...,NaN,"{'classifier': RandomForestClassifier(), 'clas...",0.921277,0.931486,0.934632,0.928863,0.922062,0.927664,0.005230,4
300,0.013308,0.001449,0.977479,0.063108,KNeighborsClassifier(),NaN,NaN,NaN,NaN,NaN,...,NaN,"{'classifier': KNeighborsClassifier(), 'classi...",0.927546,0.931220,0.929415,0.924166,0.922990,0.927067,0.003099,5


As seen in the 'best_model.score(X_test, y_test)' call, the Support Vector Machine is the best model for the problem with an f1-macro of 93%.